# Import Library

In [83]:
# This model is based on (content-based filtering) identifying similarities between skills and job descriptions. 
#We utilized TF-IDF to analyze text based on content for keywords. 
#We utilized (Doc2vec) to learn vector representations of material for similarity comparisons.
#We utilized (BERT).

In [ ]:
!pip3 install -U sentence-transformers

In [84]:
import pandas as pd
import numpy as np
import re
import string
import heapq
import nltk 

from nltk.tokenize import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
toktok = ToktokTokenizer()
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
jobs_df = pd.read_csv('JobsData.csv')
jobs_df

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function:,Industry:,Specialization:,Qualification:,Hiring Location:,Role:,Vacancies:
0,Python Engineer,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN,NaN,NaN
1,PYTHON DEVELOPER,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,NaN,NaN
2,Python Developer,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",Posted 6 days ago,5 - 8 yrs,Mumbai,http://www.innovatorwebsolutions.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,NaN,NaN
3,Python - Odoo,AxisTechnolabs,Responsibilities : We are looking for Freshers...,"python,django,itskills,html5,api,jquery",Posted 6 days ago,0 - 1 yrs,Ahmedabad,http://www.axistechnolabs.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,NaN,NaN
4,Python Developer,pearl global solutions,"Python Developer Django Job Type : Full time,...","python,database,django,teamplayer,sql",Posted 6 days ago,4 - 7 yrs,Cochin/ Kochi/ Ernakulam,https://pearlglobalsolutions.com/,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Planning and Performance Management Analyst,Armaco Chemical Processes Systems Pvt Ltd,Description: Aramco energizes the world econom...,"reports,managementaccounting,reporting,cfa,exc...",Posted few days ago,1 - 3 yrs,Jeddah,http://www.armacocps.com/,Accounting/Tax/Company Secretary/Audit,"Manufacturing/Industrial, Petrochemicals",Bookkeeping,MBA/ PGDM\r\n \r\n ...,NaN,NaN,NaN
2246,Cluster Manager - SALPL - Prime,Bajaj Finserv Lending Ltd,Cluster Manager - SALPL - Prime Location HYDER...,"msexcel,unicaandsalesforce,communication,motiv...",Posted few days ago,2 - 5 yrs,Hyderabad/Secunderabad,http://www.bajajfinserv.in/finance/,IT Software : Software Products & Services,Financial Services/Stockbroking,Software Engineer,ME/ M.Tech./ MS (Engg/ Sciences)\r\n ...,NaN,NaN,NaN
2247,Workforce Analyst,MSNU Recruiters\r\n (More Jobs),"Hello all,\r\nOne of the best opportunity for ...","DataAnalysis,teamleader,projectmanager,project...",Posted 5 days ago,1 - 4 yrs,(More Jobs),NaN,Operations/Customer Service/Telecalling/Backend,"Consulting Services, Internet/Dot com/ISP",Non-Voice Process - Domestic BPO Type,Any Graduate,NaN,"Data Entry/ DTP/ Data Conversion - Operator, D...",NaN
2248,Accounting Trainee | D A K S M And Company LLP,Anitej Services Pvt Ltd,"About the job Looking for smart, vibrant and e...","msexcel,tally,accounting,internship,finance,ta...",Posted few days ago,0 - 3 yrs,Noida/Greater Noida,http://www.cos.youth4work.com,Accounting/Tax/Company Secretary/Audit,IT-Software,ICWA,ME/ M.Tech./ MS (Engg/ Sciences)\r\n ...,NaN,NaN,NaN


# dataset Preprocessing


In [86]:
jobs_df.shape

(2250, 15)

In [87]:
jobs_df.columns

Index(['Job Name', 'Company Name', 'JD', 'Skills', 'Date Posted', 'YOE',
       'Location', 'Website', 'Job Function:', 'Industry:', 'Specialization:',
       'Qualification:', 'Hiring Location:', 'Role:', 'Vacancies:'],
      dtype='object')

In [88]:
jobs_df.isnull().sum()

Job Name               0
Company Name           0
JD                     0
Skills                 0
Date Posted            0
YOE                    0
Location              71
Website              296
Job Function:          0
Industry:              0
Specialization:        0
Qualification:         0
Hiring Location:    2244
Role:               2163
Vacancies:          2238
dtype: int64

In [89]:
jobs_df=jobs_df.fillna(' ')

In [90]:
jobs_df.isnull().sum()

Job Name            0
Company Name        0
JD                  0
Skills              0
Date Posted         0
YOE                 0
Location            0
Website             0
Job Function:       0
Industry:           0
Specialization:     0
Qualification:      0
Hiring Location:    0
Role:               0
Vacancies:          0
dtype: int64

In [91]:
jobs_df['Job Name']=jobs_df['Job Name']+['Specialization:']+['Jop Function:']

In [92]:
# Drop multiple columns
columns_to_drop = ['Vacancies:', 'Role:', 'Hiring Location:', 'Qualification:','Industry:','Specialization:','Job Function:','Website','Location','Date Posted']
jobs_df = jobs_df.drop(columns=columns_to_drop, axis=1)


In [93]:
jobs_df.head()

,Job Name,Company Name,JD,Skills,YOE
0,Python EngineerSpecialization:Jop Function:,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",2 - 5 yrs
1,PYTHON DEVELOPERSpecialization:Jop Function:,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",4 - 7 yrs
2,Python DeveloperSpecialization:Jop Function:,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",5 - 8 yrs
3,Python - OdooSpecialization:Jop Function:,AxisTechnolabs,Responsibilities : We are looking for Freshers...,"python,django,itskills,html5,api,jquery",0 - 1 yrs
4,Python DeveloperSpecialization:Jop Function:,pearl global solutions,"Python Developer Django Job Type : Full time,...","python,database,django,teamplayer,sql",4 - 7 yrs


In [94]:
jobs_df['full_text'] = jobs_df['Job Name'] + ' ' + jobs_df['JD'] + ' ' + jobs_df['Company Name'] + ' ' + jobs_df['Skills']
jobs_df['raw_full_text'] = jobs_df['Job Name'] + ' ' + jobs_df['JD'] + ' ' + jobs_df['Company Name'] + ' ' + jobs_df['Skills']


In [95]:
df = jobs_df[['full_text', 'raw_full_text']].copy()


# Text preprocessing

In [96]:
# products preprocessing
#1-Replaces specific HTML entities like "br", or empty strings.
#2-Uses a regular expression to remove HTML tags (text enclosed in '<' and '>').
#3-Replaces consecutive whitespaces with a single space.
def cleanhtml(raw_html):
    '''
    Input: HTML Text
    Output: Cleaned HTML Text
    <tag> </tag> removed and other cleaning
    '''
    cleantext = ''
    try:
        raw_html = raw_html.replace(" br ", " ")
        raw_html = raw_html.replace("nbsp"," ")
        raw_html = raw_html.replace("ndash"," ")
        raw_html = raw_html.replace("&rsquo;", ' ')
        raw_html = raw_html.replace("&trade;", ' ')
        raw_html = raw_html.replace("&amp",' ')
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', raw_html)
        cleantext = ' '.join(cleantext.split())
    except:
        print(raw_html)
    return cleantext


In [97]:
#cleaning and preparing text data
#1- convert the text to lowercase
#2-removes punctuation from the txet 
#3- split the text into a list of word 
#4-Joins the list of words back into a single string with words separated by a single space.
def text_preprocessing(text):
    return ' '.join(text.lower().translate(str.maketrans('', '', string.punctuation)).split())

df['full_text'] = df['full_text'].apply(text_preprocessing)
df['full_text'] = np.vectorize(cleanhtml)(df['full_text'])

In [98]:
df['full_text'][2]

'python developerspecializationjop function job category developmentjob type full timejob location malad wwe are looking for a python web developer responsible for managing the interchange of data between the server and the usersgood to have a knowledge of django web frameworkknowledge of python django framework understanding of serverside scripting languagesstrong experience using web services and apis in pythonhands on python django experience esp with mysql django rest frameworkminimum 5 years experience in designing and developing applications using python innovator web solutions pvtltd restpythondjangowebdevelopermysqlapi'

In [99]:
def _lemma(text):
    try:
        tokens = [lemma.lemmatize(token) for token in toktok.tokenize(text) if token not in stop_words]
        lemmatized_text = ' '.join(tokens)
    except:
        print(text)
    return lemmatized_text

df['full_text'] = np.vectorize(_lemma)(df['full_text'])


In [100]:
df

,full_text,raw_full_text
0,python engineerspecializationjop function jobd...,Python EngineerSpecialization:Jop Function: jo...
1,python developerspecializationjop function pyt...,PYTHON DEVELOPERSpecialization:Jop Function: P...
2,python developerspecializationjop function job...,Python DeveloperSpecialization:Jop Function: J...
3,python odoospecializationjop function responsi...,Python - OdooSpecialization:Jop Function: Resp...
4,python developerspecializationjop function pyt...,Python DeveloperSpecialization:Jop Function: P...
...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...
2246,cluster manager salpl primespecializationjop f...,Cluster Manager - SALPL - PrimeSpecialization:...
2247,workforce analystspecializationjop function he...,Workforce AnalystSpecialization:Jop Function: ...
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...


# Iteration1 :  TF-IDF



#Lexical Similarity Model
 
#Based on Syntactic Similarity


In [101]:
# Initialize TfidfVectorizer
tfidf = TfidfVectorizer(
    token_pattern='(?u)\\b\\w\\w+\\b', # Regular expression for word tokens
    ngram_range=(1, 2),  # Consider unigrams and bigrams
    stop_words='english',  # Removes common English stop words
    analyzer='word'  # Determines whether to use word-level analysis.
)

# Use fit_transform to transform the 'full_text' column into a matrix of TF-IDF features.
features = tfidf.fit_transform(df['full_text'].values.tolist())

# Create a DataFrame with TF-IDF features
df_tfidf = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names_out())


In [102]:
# Calculate cosine similarity matrix for the TF-IDF features
similarity_matrix = cosine_similarity(df_tfidf, df_tfidf)
similarity_matrix.shape

(2250, 2250)

In [103]:
# top 10 similar items to an given item
#representing similarity scores between a specific item and other items in the dataset. (index,similarity score)
#sorting based on the second element of each pair(index,similarity score)
heapq.nlargest(10, enumerate(similarity_matrix[0]), key=lambda x: x[1])

[(0, 1.0000000000000002),
 (2015, 0.12351590456101642),
 (123, 0.08567458920552229),
 (191, 0.07277007697602787),
 (68, 0.06896368957606744),
 (192, 0.06483867833853718),
 (106, 0.060276136970577675),
 (501, 0.05790885158924269),
 (1461, 0.057741249085518025),
 (1457, 0.055989033613848875)]

In [104]:
df['recommendation_list_tfidf'] = df.index.map(
    lambda indx: [a for a, b in heapq.nlargest(
        10, enumerate(similarity_matrix[indx]), key=lambda x: x[1])][1:])

In [105]:
#add new column named (recommendation_list_tfidf) to the dataframe df.
#applies a function to each index of the Dataframe --->df.index.map(...).
#lmabda indx:--->defines an anonymous function that takes an index indx as input.
#[1:] removes the first item (index 0) from the list, as it likely represents the item itself.


In [106]:
#The code iterates through each item in the DataFrame.
#For each item, it finds the 10 most similar items based on the similarity_matrix.
#It stores the indices of those 10 similar items (excluding the item itself) in the recommendation_list_tfidf column.

In [107]:
df

,full_text,raw_full_text,recommendation_list_tfidf
0,python engineerspecializationjop function jobd...,Python EngineerSpecialization:Jop Function: jo...,"[2015, 123, 191, 68, 192, 106, 501, 1461, 1457]"
1,python developerspecializationjop function pyt...,PYTHON DEVELOPERSpecialization:Jop Function: P...,"[2, 29, 12, 55, 7, 88, 199, 67, 70]"
2,python developerspecializationjop function job...,Python DeveloperSpecialization:Jop Function: J...,"[70, 88, 126, 24, 25, 199, 1, 113, 204]"
3,python odoospecializationjop function responsi...,Python - OdooSpecialization:Jop Function: Resp...,"[15, 1886, 818, 1115, 94, 180, 776, 1477, 645]"
4,python developerspecializationjop function pyt...,Python DeveloperSpecialization:Jop Function: P...,"[2, 199, 70, 104, 105, 88, 12, 23, 18]"
...,...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...,"[2026, 2231, 2105, 2127, 2104, 2227, 2114, 197..."
2246,cluster manager salpl primespecializationjop f...,Cluster Manager - SALPL - PrimeSpecialization:...,"[2249, 1111, 1112, 1113, 498, 499, 2108, 1748,..."
2247,workforce analystspecializationjop function he...,Workforce AnalystSpecialization:Jop Function: ...,"[2234, 2156, 460, 461, 462, 463, 464, 468, 469]"
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...,"[1007, 818, 1115, 180, 776, 645, 1477, 496, 94]"


# iteration 2:Text Similarty based on Semantics


In [108]:
#Doc2Vec: This class is used for creating and training Doc2Vec models, which learn to represent documents as dense numerical vectors.
#TaggedDocument: This class is used to prepare text documents for training a Doc2Vec model. It associates each document with a unique tag or label.
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [109]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df.full_text.values.tolist())]

In [110]:
model = Doc2Vec(documents, vector_size=40, window=3, min_count=2, workers=4, epochs=50, negative= 5, seed = 0)

In [111]:
model.save("d2v.model")
model.dv.most_similar(0, topn=10)

[(1512, 0.6466156244277954),
 (1246, 0.6149023175239563),
 (125, 0.6007382869720459),
 (2039, 0.5966715812683105),
 (897, 0.5841128826141357),
 (1778, 0.567641019821167),
 (762, 0.5600676536560059),
 (1149, 0.5593886971473694),
 (816, 0.5418993830680847),
 (138, 0.541480541229248)]

In [112]:
print (documents[0])
print (documents[1810])

TaggedDocument<python engineerspecializationjop function jobdescription 2 year experience working python strong analytical skill ability churn scripting solution quick turnaround time inclination learn new technology preferably background knowledge data science machine learning hadoop east india security ltd pythonhadoopmachinelearning, [0]>
TaggedDocument<android developerspecializationjop function position android developer department mobile application development vacancy 2 vacancy type permanent location jodhpur rajasthan education btechbe mtech bscmsc computer science bcamca experience 2 year joining time immediately technical skill good knowledge android sdk kotin parsing jsonxml sqlite database local database management additional knowledge knowledge web communication service shall preferable like xml json parsing v2r solution databasemanagementdatabasexmljsontechnicalskills, [1810]>


In [113]:
#Initializing a New Column
df['recommendation_list_Doc2Vec'] = None

In [114]:
#Generating Recommendations for Each Job
for indx in df.index:
    df.at[indx, 'recommendation_list_Doc2Vec'] = [item for item, score in model.dv.most_similar([indx], topn=10)]

In [115]:
df

,full_text,raw_full_text,recommendation_list_tfidf,recommendation_list_Doc2Vec
0,python engineerspecializationjop function jobd...,Python EngineerSpecialization:Jop Function: jo...,"[2015, 123, 191, 68, 192, 106, 501, 1461, 1457]","[1512, 1246, 125, 2039, 897, 1778, 762, 1149, ..."
1,python developerspecializationjop function pyt...,PYTHON DEVELOPERSpecialization:Jop Function: P...,"[2, 29, 12, 55, 7, 88, 199, 67, 70]","[4, 1842, 25, 1835, 1844, 1382, 1246, 1866, 19..."
2,python developerspecializationjop function job...,Python DeveloperSpecialization:Jop Function: J...,"[70, 88, 126, 24, 25, 199, 1, 113, 204]","[985, 1838, 16, 895, 1545, 567, 18, 413, 3, 37]"
3,python odoospecializationjop function responsi...,Python - OdooSpecialization:Jop Function: Resp...,"[15, 1886, 818, 1115, 94, 180, 776, 1477, 645]","[1238, 37, 1839, 2, 1861, 1989, 895, 539, 584,..."
4,python developerspecializationjop function pyt...,Python DeveloperSpecialization:Jop Function: P...,"[2, 199, 70, 104, 105, 88, 12, 23, 18]","[1, 1842, 1788, 1835, 1844, 97, 158, 1884, 316..."
...,...,...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...,"[2026, 2231, 2105, 2127, 2104, 2227, 2114, 197...","[2036, 2231, 1596, 2027, 404, 407, 2010, 2013,..."
2246,cluster manager salpl primespecializationjop f...,Cluster Manager - SALPL - PrimeSpecialization:...,"[2249, 1111, 1112, 1113, 498, 499, 2108, 1748,...","[746, 257, 914, 1832, 274, 1697, 1782, 1789, 1..."
2247,workforce analystspecializationjop function he...,Workforce AnalystSpecialization:Jop Function: ...,"[2234, 2156, 460, 461, 462, 463, 464, 468, 469]","[1744, 82, 1473, 646, 1224, 1316, 337, 2186, 1..."
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...,"[1007, 818, 1115, 180, 776, 645, 1477, 496, 94]","[431, 1918, 945, 1919, 1117, 438, 470, 461, 19..."


## Iteration3: Text similarity with Contextual Embedding (BERT)

In [116]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

## Bert is a contextual algorithm it doesnot require Text Preprocessing and Lemma
corpus = df.raw_full_text.values.tolist()

In [117]:
# encode corpus
sentence_embeddings = model.encode(corpus)

In [118]:
sentence_embeddings.shape

(2250, 768)

In [119]:
similarity_matrix_bert = cosine_similarity(sentence_embeddings, sentence_embeddings)

In [120]:
df['recommendation_list_bert'] = None
topK = 10
for indx in df.index:
    df.at[indx, 'recommendation_list_bert'] = [a for a,b in heapq.nlargest(topK+1, enumerate(similarity_matrix[indx]), key=lambda x: x[1])][1:]


In [121]:
df

,full_text,raw_full_text,recommendation_list_tfidf,recommendation_list_Doc2Vec,recommendation_list_bert
0,python engineerspecializationjop function jobd...,Python EngineerSpecialization:Jop Function: jo...,"[2015, 123, 191, 68, 192, 106, 501, 1461, 1457]","[1512, 1246, 125, 2039, 897, 1778, 762, 1149, ...","[2015, 123, 191, 68, 192, 106, 501, 1461, 1457..."
1,python developerspecializationjop function pyt...,PYTHON DEVELOPERSpecialization:Jop Function: P...,"[2, 29, 12, 55, 7, 88, 199, 67, 70]","[4, 1842, 25, 1835, 1844, 1382, 1246, 1866, 19...","[2, 29, 12, 55, 7, 88, 199, 67, 70, 151]"
2,python developerspecializationjop function job...,Python DeveloperSpecialization:Jop Function: J...,"[70, 88, 126, 24, 25, 199, 1, 113, 204]","[985, 1838, 16, 895, 1545, 567, 18, 413, 3, 37]","[70, 88, 126, 24, 25, 199, 1, 113, 204, 124]"
3,python odoospecializationjop function responsi...,Python - OdooSpecialization:Jop Function: Resp...,"[15, 1886, 818, 1115, 94, 180, 776, 1477, 645]","[1238, 37, 1839, 2, 1861, 1989, 895, 539, 584,...","[15, 1886, 818, 1115, 94, 180, 776, 1477, 645,..."
4,python developerspecializationjop function pyt...,Python DeveloperSpecialization:Jop Function: P...,"[2, 199, 70, 104, 105, 88, 12, 23, 18]","[1, 1842, 1788, 1835, 1844, 97, 158, 1884, 316...","[2, 199, 70, 104, 105, 88, 12, 23, 18, 99]"
...,...,...,...,...,...
2245,planning performance management analystspecial...,Planning and Performance Management AnalystSpe...,"[2026, 2231, 2105, 2127, 2104, 2227, 2114, 197...","[2036, 2231, 1596, 2027, 404, 407, 2010, 2013,...","[2026, 2231, 2105, 2127, 2104, 2227, 2114, 197..."
2246,cluster manager salpl primespecializationjop f...,Cluster Manager - SALPL - PrimeSpecialization:...,"[2249, 1111, 1112, 1113, 498, 499, 2108, 1748,...","[746, 257, 914, 1832, 274, 1697, 1782, 1789, 1...","[2249, 1111, 1112, 1113, 498, 499, 2108, 1748,..."
2247,workforce analystspecializationjop function he...,Workforce AnalystSpecialization:Jop Function: ...,"[2234, 2156, 460, 461, 462, 463, 464, 468, 469]","[1744, 82, 1473, 646, 1224, 1316, 337, 2186, 1...","[2234, 2156, 460, 461, 462, 463, 464, 468, 469..."
2248,accounting trainee k company llpspecialization...,Accounting Trainee | D A K S M And Company LLP...,"[1007, 818, 1115, 180, 776, 645, 1477, 496, 94]","[431, 1918, 945, 1919, 1117, 438, 470, 461, 19...","[1007, 818, 1115, 180, 776, 645, 1477, 496, 94..."
